In [1]:
from __future__ import print_function
"""
Tutorial: A reference implementation of non-hermitian cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# Non-Hermitian Cavity Quantum Electrodynamics Configuration Interaction Singles (NH-CQED-CIS) 

## I. Theoretical Overview

This tutorial builds from the Psi4Numpy tutorial on canonical CIS for molecular systems, that implements the configuration interaction singles method in the spin orbital notation. The groundwork for working in the spin orbital notation has been laid out in "Introduction to the Spin Orbital Formulation of Post-HF methods" [tutorial](../08_CEPA0_and_CCD/8a_Intro_to_spin_orbital_postHF.ipynb). It is highly recommended to work through that introduction before starting this tutorial. 

We propose a novel formulation of ab initio CQED that couples an ab initio molecular Hamiltonian to a non-Hermitian photonic Hamiltonian to capture the finite lifetime and energy uncertainty of nanoconfined optical resonances.   As a first realization of this theory, will develop a configuration interaction singles approach for the energy eigenstates of this non-Hermitian polaritonic Hamiltonian, which will allow us to leverage tools of modern quantum chemistry, including analytic evaluation of forces and nonadiabatic couplings, to study polaritonic structure and reactivity.  This will pave the way for future developments of a hierarchy of CI-based approaches that can be systematically improved in terms of their accuracy, and are also interoperable with the powerful machinery of quantum chemistry codes.  In particular, we envision that Complete Active Space CI theory, which has been a particularly promising approach for simulating photochemistry, can be merged with NH-CQED to provide a powerful tool for simulating polaritonic chemistry.
In the following presentation of NH-CQED-CIS theory, we will consider only a single photonic degree of freedom for notational simplicity, but generalizations to additional photonic modes is feasible.  The total polaritonic Hamiltonian operator may be written as
\begin{equation}
    \hat{H} = \sum_{pq} h_{pq} \hat{a}_p^{\dagger} \hat{a}_q + 
    \sum_{pqrs} V_{pq}^{rs} \hat{a}_p^{\dagger} \hat{a}_q^{\dagger} \hat{a}_s \hat{a}_r
    + \hbar \left( \omega - i \frac{\gamma}{2} \right)\hat{b}^{\dagger} \hat{b} 
    + \sum_{pq} \hbar g_{pq} \hat{a}_p^{\dagger} \hat{a}_q \left(\hat{b}^{\dagger} + \hat{b} \right),
\end{equation}
where $h_{pq}$ includes the molecular electronic kinetic energy and electron-nuclear potential
integrals, $V_{pq}^{rs}$ denotes the molecular 2-electron repulsion integrals, 
$\hat{a}_p^{\dagger}$ and $\hat{a}_q$ denotes the electronic creation and annihilation 
operators, $\omega$ and $\gamma$ denote the energy and lifetime of the photonic degree of 
freedom, $\hat{b}^{\dagger}$ and $\hat{b}$ denote the photonic raising and lowering operators, and $g_{pq}$ denotes the coupling between the photonic and electronic degrees of freedom. 

In [1]:
# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la

# ==> Set Basic Psi4 Options <==

# Memory specifications
psi4.set_memory(int(2e9))
numpy_memory = 2

# Output options
psi4.core.set_output_file('output.dat', False)

Define the molecule!

In [35]:
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf'})

# also define photon energy as 0.25 a.u. which is about 6.8 eV
E_phot = 0.25

We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e` and `scf_wfn`. We also check the memory requirements for computation:

In [9]:
# Get the SCF wavefunction & energies
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

# Check memory requirements
nmo = scf_wfn.nmo()
I_size = (nmo**4) * 8e-9
print('\nSize of the ERI tensor will be %4.2f GB.\n' % I_size)
memory_footprint = I_size * 1.5
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted \
                     memory limit of %4.2f GB." % (memory_footprint, numpy_memory))


Size of the ERI tensor will be 0.00 GB.



We first obtain orbital information from our wavefunction. We also create an instance of MintsHelper to help build our molecular integrals:

In [10]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Get basis and orbital information
nbf = mints.nbf()          # Number of basis functions
nalpha = wfn.nalpha()  # Number of alpha electrons
nbeta = wfn.nbeta()    # Number of beta electrons
nocc = nalpha + nbeta      # Total number of electrons
nso = 2 * nbf              # Total number of spin orbitals
nvirt = nso - nocc         # Number of virtual orbitals

Let's first take care of the molecule-photon coupling terms.  A 
dipole-dipole coupling potential provides a reasonable approximation to this term:
\begin{equation}
g_{pq} = \frac{1}{r^3} \mu_{pq} \cdot \mu_s - \frac{3}{r^5}\left(\mu_{pq} \cdot r \right) \left(\mu_s \cdot r \right),
\end{equation}
where $\mu_{pq}$ are the the dipole integrals in the MO basis.

We will need to build a matrix of these coupling elements that matches the spin-blocking used in the subsequent CIS Hamiltonian.

In [11]:
# ==> array of dipole integrals <==
# start by assuming photon polarized along z so only
# need z-component of molecular dipole integrals!
mu_z = np.asarray(mints.ao_dipole()[2])

# Using np.kron, we project mu_z into the space of the 2x2 identity
# The result is the dipole contribution to the Hamiltonian in the spin orbital formulation
mu_z_mat = np.kron(np.eye(2), mu_z)
print(mu_z_mat)

[[ 0.14322583  0.03390212 -0.05079193  0.          0.          0.00375868
   0.00375868  0.          0.         -0.          0.          0.
   0.          0.        ]
 [ 0.03390212  0.14322583 -0.64117284  0.          0.         -0.14997188
  -0.14997188  0.          0.         -0.          0.          0.
  -0.         -0.        ]
 [-0.05079193 -0.64117284  0.14322583  0.          0.         -0.33409068
  -0.33409068 -0.         -0.          0.          0.          0.
  -0.         -0.        ]
 [ 0.          0.          0.          0.14322583  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.14322583  0.10895216
  -0.10895216  0.          0.          0.          0.          0.
   0.         -0.        ]
 [ 0.00375868 -0.14997188 -0.33409068  0.          0.10895216 -1.1365489
  -0.20657896  0.         -0.         -0.          0.          0.
  -0.         -0.        ]
 [ 0.00

Form transformation vectors from AO->MO basis!

In [22]:
# Get orbital energies, cast into NumPy array, and extend eigenvalues
eps_a = np.asarray(wfn.epsilon_a())
eps_b = np.asarray(wfn.epsilon_b())
eps = np.append(eps_a, eps_b)

# Get coefficients, block, and sort
Ca = np.asarray(wfn.Ca())
Cb = np.asarray(wfn.Cb())
C = np.block([
             [      Ca,         np.zeros_like(Cb)],
             [np.zeros_like(Ca),          Cb     ]])


# Sort the columns of C according to the order of orbital energies
C = C[:, eps.argsort()]

# Sort orbital energies
eps = np.sort(eps)

def ao_to_mo(hao, C):
    '''
    Transform hao, which is the core Hamiltonian in the spin orbital basis,
    into the MO basis using MO coefficients
    '''
    
    return np.einsum('pQ, pP -> PQ', 
           np.einsum('pq, qQ -> pQ', hao, C, optimize=True), C, optimize=True)

# Transform mu_z_so and hao into MO basis
mu_z_so = ao_to_mo(mu_z_mat, C)


Restricting the polarization of the photon to the z-direction means we need 
only worry about the scalar separation (along the z-axis) between the center-of-mass of
the molecular transition dipoles and the photon transition dipole ($r_z$) and the 
$z$-component of the photon transition dipole $\mu_{s,z}$.
\begin{equation}
g_{pq} = \frac{1}{r_z^3} \mu_{z,pq} \cdot \mu_{z,s} - \frac{3}{r_z^5}\left(\mu_{z,pq} \cdot r_z \right) \left(\mu_{s,z} \cdot r_z \right),
\end{equation}

In [36]:
''' build g matrix here in the same shape as the dipole matrix and the C matrix'''
# first define the constant terms that contribute to the matrix

# tdm of the photonic mode in atomic units
mu_s = 100.0

# separation between the photon com and molecule com in atomic units
r = 50.0

# 1/r^3 term
oer3 = 1 / r ** 3

# 3/r^5 term
toer5 = 3 / r ** 5

# product between photonic tdm and separation vector component r
mus_r = mu_s * r

# contribution to g-matrix from first term
g_one = oer3 * mu_z_so * mu_s

# contribution to g-matrix from second term
g_two = toer5 * mu_z_so * r * mus_r

g = g_one - g_two

     

## NH-CQED-CIS

In CIS, only single excitations from the occupied (indices i,j,k...) to the virtual (indices a,b,c...) orbitals are included. As a result, CIS gives transition energies to an excited state. 

The polaritonic energy eigenfunctions for state $I$ in the
CQED-CIS ansatz can be written as 
\begin{equation}
\Psi_I = c_{0,0} |\Phi_0\rangle |0\rangle + 
c_{0,1} |\Phi_0\rangle |1\rangle +
c^a_{i,0} |\Phi_i^a\rangle |0\rangle +
c^a_{i,1} |\Phi_i^a\rangle |1\rangle 
\end{equation}

Assuming we are using canonical Hartree-Fock spin orbitals($\{\mathrm{\psi_p}\}$) with orbital energies $\{\epsilon_p\}$, we can build a shifted CIS Hamiltonian matrix:

\begin{equation}
\tilde{\textbf{H}} = \textbf{H} - E_0 \textbf{I} = 
[\langle s| \langle \Phi_P | \hat{H} - E_0|\Phi_Q \rangle | t \rangle ]
\end{equation}
where
\begin{equation}
\Phi_P, \Phi_Q \in {\Phi_i^a} \; {\rm and} \; |s\rangle, |t\rangle \in {|0\rangle, |1\rangle}
\end{equation}

and where $E_0$ is the ground state Hartree-Fock state energy given by $\langle \Phi | \hat{H}_e|\Phi \rangle$,
where $\hat{H}_e$ contains only the molecular electronic terms.

The matrix elements of this shifted CIS Hamiltonian matrix contains three contributions:

\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H} - E_0|\Phi_j^b \rangle | t \rangle = 
\langle s | \langle \Phi_i^a | \hat{H}_e - E_0|\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle.
\end{equation}
The first term is similar to the ordinary CIS matrix elements with the requirement that the photonic
bra and ket states be identical:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H_e} - E_0|\Phi_j^b \rangle | t \rangle = \left((\epsilon_a - \epsilon_i)\delta_{ij} \delta_{ab}
+ \langle aj || ib \rangle \right) \delta_{st}.
\end{equation}
The second term vanishes unless both the photonic and molecular bra and ket states are identical:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle = \delta_{st} \delta_{ij} \delta_{ab} \left( \hbar \omega + i\frac{\gamma}{2}\right)\sqrt{s}.
\end{equation}
The third term couples particular transitions between photonic and molecular bra and ket states:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle =
\left( 1 - \delta_{st} \right) \left( g_{ab} \delta_{ij} - g_{ij} \delta_{ab} \right)
\end{equation}

This then becomes a standard eigenvalue equation from which we can solve for the excitation energies and the wavefunction expansion coefficients:

\begin{equation}
\tilde{\textbf{H}} \textbf{c}_K = \Delta E_K\textbf{c}_K, \,\Delta E_K = E_K - E_0
\end{equation}



## II. Implementation

As with previous tutorials, let's begin by importing Psi4 and NumPy and setting memory and output file options.

We now define the molecule and set Psi4 options:

We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e and scf_wfn`. We also check the memory requirements for computation:

We now build our 2-electron integral, a 4D tensor, in the spin orbital formulation. We also convert it into physicist's notation and antisymmetrize for easier manipulation of the tensor later on. 

In [37]:
def spin_block_tei(I):
    '''
    Spin blocks 2-electron integrals
    Using np.kron, we project I and I tranpose into the space of the 2x2 ide
    The result is our 2-electron integral tensor in spin orbital notation
     '''
    identity = np.eye(2)
    I = np.kron(identity, I)
    return np.kron(identity, I.T)
 
I = np.asarray(mints.ao_eri())
I_spinblock = spin_block_tei(I)
 
# Convert chemist's notation to physicist's notation, and antisymmetrize
# (pq | rs) ---> <pr | qs>
# <pr||qs> = <pr | qs> - <pr | sq>
gao = I_spinblock.transpose(0, 2, 1, 3) - I_spinblock.transpose(0, 2, 3, 1)

We get the orbital energies from alpha and beta electrons and append them together. We spin-block the coefficients obtained from the reference wavefunction and convert them into NumPy arrays. There is a set corresponding to coefficients from alpha electrons and a set of coefficients from beta electrons. We then sort them according to the order of the orbital energies using argsort():

We now transform the 2-electron integral from the AO basis into the MO basis using the coefficients:


In [38]:
# Transform gao, which is the spin-blocked 4d array of physicist's notation,
# antisymmetric two-electron integrals, into the MO basis using MO coefficients
gmo = np.einsum('pQRS, pP -> PQRS',
      np.einsum('pqRS, qQ -> pQRS',
      np.einsum('pqrS, rR -> pqRS',
      np.einsum('pqrs, sS -> pqrS', gao, C, optimize=True), C, optimize=True), C, optimize=True), C, optimize=True)

Now that we have our integrals, coefficents, and orbital energies set up in with spin orbitals, we can start our CIS procedure. We first start by initializing the shifted Hamiltonion matrix $\tilde{\textbf{H}}$ (`HCIS`). Let's think about the size of $\tilde{\textbf{H}}$. We need all possible single excitations from the occupied to virtual orbitals. This is given by the number of occupied orbitals times the number of virtual orbitals times the number
of photonic states (`nocc * nvirt * nphot`).

The size of our matrix is thus `nocc * nvirt * nphot` by `nocc * nvirt * nphot`. 

In [39]:
# Initialize CIS matrix.
# With the number of photonic states limited to 2,
# i.e. the |0> state and the |1> state, 
# the dimensions are the number of possible single excitations * 2
nphot = 2
HCIS = np.zeros((nocc * nvirt * nphot, nocc * nvirt * nphot))

Next, we want to build all possible excitations from occupied to virtual orbitals. We create two for-loops that will loop over the number of occupied orbitals and number of virtual orbitals, respectively, and store the combination of occupied and virtual indices as a tuple `(i, a)`. We put all tuples in a list called `excitations`. 

In [40]:
# Build the possible excitations, collect indices into a list
excitations = []
for i in range(nocc):
    for a in range(nocc, nso):
        for s in range(0,2):
            excitations.append((i, a, s))
        
print(excitations)



[(0, 10, 0), (0, 10, 1), (0, 11, 0), (0, 11, 1), (0, 12, 0), (0, 12, 1), (0, 13, 0), (0, 13, 1), (1, 10, 0), (1, 10, 1), (1, 11, 0), (1, 11, 1), (1, 12, 0), (1, 12, 1), (1, 13, 0), (1, 13, 1), (2, 10, 0), (2, 10, 1), (2, 11, 0), (2, 11, 1), (2, 12, 0), (2, 12, 1), (2, 13, 0), (2, 13, 1), (3, 10, 0), (3, 10, 1), (3, 11, 0), (3, 11, 1), (3, 12, 0), (3, 12, 1), (3, 13, 0), (3, 13, 1), (4, 10, 0), (4, 10, 1), (4, 11, 0), (4, 11, 1), (4, 12, 0), (4, 12, 1), (4, 13, 0), (4, 13, 1), (5, 10, 0), (5, 10, 1), (5, 11, 0), (5, 11, 1), (5, 12, 0), (5, 12, 1), (5, 13, 0), (5, 13, 1), (6, 10, 0), (6, 10, 1), (6, 11, 0), (6, 11, 1), (6, 12, 0), (6, 12, 1), (6, 13, 0), (6, 13, 1), (7, 10, 0), (7, 10, 1), (7, 11, 0), (7, 11, 1), (7, 12, 0), (7, 12, 1), (7, 13, 0), (7, 13, 1), (8, 10, 0), (8, 10, 1), (8, 11, 0), (8, 11, 1), (8, 12, 0), (8, 12, 1), (8, 13, 0), (8, 13, 1), (9, 10, 0), (9, 10, 1), (9, 11, 0), (9, 11, 1), (9, 12, 0), (9, 12, 1), (9, 13, 0), (9, 13, 1)]


Now we can evaluate the matrix elements of the shifted CIS Hamiltonian matrix using the equation given above. For each element, there are several layers of indexing that we have to consider. 
First, there are the indices of the element itself, which gives the position of the element in the matrix. Indices `p` and `q` are used:

`HCIS[p, q]`

Second, there are two sets of excitations from occupied to virtual orbitals corresponding to the bra and ket of each matrix element. For these, we will take advantage of the `excitations` list that we build with the list of all possible excitations. We will use indices i and a to denote the excitation in the bra (`left_excitation`) and j and b to denote the excitation in the ket (`right_excitation`). 

To manage these indices, we will use the `enumerate` function.
Note that `first, second in enumerate(iterable)` takes an iterable and returns a counter / index to the first variable, and the values in the iterable to the second variable.

Note that a Kronecker delta $\delta_{pq}$ can be represented as p == q. 

\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H_e} - E_0|\Phi_j^b \rangle | t \rangle = \left((\epsilon_a - \epsilon_i)\delta_{ij} \delta_{ab}
+ \langle aj || ib \rangle \right) \delta_{st}.
\end{equation}
The second term vanishes unless both the photonic and molecular bra and ket states are identical:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle = \delta_{st} \delta_{ij} \delta_{ab} \left( \hbar \omega + i\frac{\gamma}{2}\right)\sqrt{s}.
\end{equation}
The third term couples particular transitions between photonic and molecular bra and ket states:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle =
\left( 1 - \delta_{st} \right) \left( g_{ab} \delta_{ij} - g_{ij} \delta_{ab} \right)
\end{equation}


In [41]:
# Form matrix elements of shifted CIS Hamiltonian
for p, left_excitation in enumerate(excitations):
    i, a, s = left_excitation
    for q, right_excitation in enumerate(excitations):
        j, b, t = right_excitation
        H_mol = (eps[a] - eps[i]) * (i == j) * (a == b) * (s == t) + gmo[a, j, i, b] * (s == t)
        H_phot = E_phot * np.sqrt(s) * (s == t) * (i == j) * (a == b)
        H_mol_phot = (1 - (s == t)) * ( g[a,b] * (i == j) - g[i,j] * (a == b) )
         
        HCIS[p, q] = H_mol + H_phot + H_mol_phot

We now use the NumPy function `linalg.eigh` (for hermitian matrices) to diagonalize the shifted CIS Hamiltonian. This will give us the excitation energies (`ECIS`). These eigenvalues correspond to the CIS total energies for various excited states. The columns of matrix `CCIS` give us the coefficients which describe the relative contribution of each singly excited determinant to the excitation energy. 


In [43]:
# Diagonalize the shifted CIS Hamiltonian
NH_ECIS, NH_CCIS = np.linalg.eigh(HCIS)
print(NH_ECIS-ECIS)


[-1.01702158e-05 -1.01702158e-05 -1.01702158e-05  5.71632292e-02
  5.71632292e-02  5.71632292e-02  1.20266102e-02  2.15586010e-02
  2.15586010e-02  2.15586010e-02  5.00856123e-02  5.00856123e-02
  3.80488319e-02  5.96045650e-02  1.39629757e-01  1.48296319e-01
  1.48296319e-01  1.48296319e-01  1.71276688e-01  1.71276688e-01
  1.42751900e-01  1.60673952e-01  1.68533997e-01  1.68533997e-01
  1.68533997e-01  1.99920618e-01  1.78362662e-01  1.78362662e-01
  1.00843645e-01  1.10366108e-01  1.01704412e-01  1.01704412e-01
  1.30225159e-01  1.30225159e-01  1.30225159e-01  1.41025986e-01
  1.10885272e-01  2.00440332e-01  2.01236864e-01  2.01236864e-01
  2.01236864e-01  2.42137591e-01  2.50000376e-01  2.50000376e-01
  1.57737678e-01  2.50001001e-01  9.96382393e-07  1.98646088e-01
 -3.19297771e-06 -3.19297771e-06  5.13508328e-02  9.13261275e-02
  9.13261275e-02  9.13261275e-02  1.00689501e-01  1.25666351e-01
  1.58682461e-01  1.58682461e-01  1.58682461e-01  2.50002899e-01
  1.49313825e-01  1.49313

For a given excitation energy, each coefficent in the linear combination of excitations represents the amount that a particular excitation contributes to the overall excitation energy. The percentage contribution of each coefficient can be calculated by squaring the coefficent and multiplying by 100. 

In [13]:
# Percentage contributions of coefficients for each state vector
percent_contrib = np.round(CCIS**2 * 100)

In addition to excitation energies, we want to print the excitations that contribute 10% or more to the overall energy, as well as their percent contribution. 

Note that `end = ''` parameter in the print function allows us to print different sections to the same line without a line break.

In [14]:
# Print detailed information on significant excitations
print('CIS:')
for state in range(len(ECIS)):
    # Print state, energy
    print('State %3d Energy (Eh) %10.7f' % (state + 1, ECIS[state]), end=' ')
    for idx, excitation in enumerate(excitations):
        if percent_contrib[idx, state] > 10:
            i, a = excitation
            # Print percentage contribution and the excitation
            print('%4d%% %2d -> %2d' % (percent_contrib[idx, state], i, a), end=' ')
    print()

CIS:
State   1 Energy (Eh)  0.2872554  100%  9 -> 10 
State   2 Energy (Eh)  0.2872554  100%  8 -> 11 
State   3 Energy (Eh)  0.2872554   50%  8 -> 10   50%  9 -> 11 
State   4 Energy (Eh)  0.3444249   44%  6 -> 10   44%  7 -> 11 
State   5 Energy (Eh)  0.3444249   45%  6 -> 11   44%  7 -> 10 
State   6 Energy (Eh)  0.3444249   44%  6 -> 11   45%  7 -> 10 
State   7 Energy (Eh)  0.3564617   50%  8 -> 10   50%  9 -> 11 
State   8 Energy (Eh)  0.3659889   50%  8 -> 12   50%  9 -> 13 
State   9 Energy (Eh)  0.3659889   93%  8 -> 13 
State  10 Energy (Eh)  0.3659889   93%  9 -> 12 
State  11 Energy (Eh)  0.3945137   20%  4 -> 10   20%  5 -> 11   30%  6 -> 12   30%  7 -> 13 
State  12 Energy (Eh)  0.3945137   33%  4 -> 11   48%  6 -> 13   11%  7 -> 12 
State  13 Energy (Eh)  0.3945137   33%  5 -> 10   11%  6 -> 13   48%  7 -> 12 
State  14 Energy (Eh)  0.4160717   50%  8 -> 12   50%  9 -> 13 
State  15 Energy (Eh)  0.5056282   44%  6 -> 10   44%  7 -> 11 
State  16 Energy (Eh)  0.5142899   

## References
1. Background paper:
 >"Toward a systematic molecular orbital theory for excited states"
[[Foresman:1992:96](http://pubs.acs.org/doi/abs/10.1021/j100180a030)] J. B. Foresman, M. Head-Gordon, J. A. Pople, M. J. Frisch, *J. Phys. Chem.* **96**, 135 (1992).


2. Algorithms from: 
	> [[CCQC:CIS](https://github.com/CCQC/summer-program/tree/master/7)] CCQC Summer Program, "CIS" accessed with https://github.com/CCQC/summer-program/tree/master/7.
    